<a href="https://colab.research.google.com/github/kellytdunn/Distance-Calculator/blob/main/GIS_Distance_calculator_OSM_street_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare Data

In [ ]:
% pip install geopandas

#import matplotlib.pyplot as plt
import geopy.distance
from google.colab import files
import io



# New Section

In [ ]:
uploaded=files.upload()
excel_file = io.BytesIO(uploaded.get('OD data with transformed access.xlsx'))


Saving OD data with transformed access.xlsx to OD data with transformed access.xlsx


In [ ]:
from geopy.distance import vincenty
import pandas as pd
import numpy as np

In [ ]:
df1 = pd.read_excel(excel_file, 0)

In [ ]:
df = df1

In [ ]:
# convert NAs to zero now, before they become the tuples I need. 

cols_fillna = ['origin_lat','origin_lng','destination_lat','destination_lng','board_stop_lat','board_stop_lon']
# replace 'NaN' with zero in these columns
for col in cols_fillna:
     df[col].fillna(1,inplace=True)

#should not see any nans in the output for the columns above
#But I think maybe the inputs were already zeroes, so they're not getting switched to 1s here

In [ ]:
df['board_stop_lon'].isnull().sum()

0

# Calculate Origin-Destination Distances

In [ ]:
import requests
import json

In [ ]:
testdf = df[:50]

In [ ]:
def Accessdistancer(row):
  lon_1 = (row['origin_lng'])
  lat_1 = (row['origin_lat'])
  lon_2 = (row['board_stop_lon'])
  lat_2 = (row['board_stop_lat'])
  # call the OSMR API
  r = requests.get(f"http://router.project-osrm.org/route/v1/foot/{lon_1},{lat_1};{lon_2},{lat_2}?overview=false""")
  # then you load the response using the json libray
  routes = json.loads(r.content) #returns dictionary which includes `routes` as one key and a list of dictionaries as its value. Diff results if fed (0,0) point.
  if "routes" in routes: 
    route_1 = routes.get("routes")[0] #takes the value from `route`, which is a list of dictionaries, then returns the first (and only) dictionary from that list. 
  else:
    route_1 = {'distance':0}
  return (route_1['distance']*0.000621371) #takes just the value of distance from the dictionary and converts meters to miles
#http://project-osrm.org/docs/v5.5.1/api/#route-service

In [ ]:
#will overwrite excel file last exported, so download it first.
current_chunk = df[14001:16501] #end +1 higher than you want
access = current_chunk.apply(Accessdistancer, axis = 1) #generate pandas series of access distances
access = pd.Series(access, name = "Street Network Distance") #give it a name
newdf = df.merge(access, left_index=True, right_index=True) #join series to existing df, based on index. just keep those in the chunk I want.
newdf.to_excel('OD dataset with street network distances.xlsx') #export df to excel

In [ ]:
#will need to re-import aggregated file before finishing the part below. 

In [ ]:
#now watch for obs where boarding location is 0 but origin is not, so a very large distance is calculated.
too_high = df['Access Distance']>7000
sum(too_high)

In [ ]:
df.loc[df['Access Distance'] > 7000, 'Access Distance'] = 0

In [ ]:
df[6704:6708]

In [ ]:
df.to_excel('OD dataset with street network distances.xlsx', index=False)